<a href="https://colab.research.google.com/github/perlatomdpi/End-Point/blob/main/1_Define_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Define the Machine Learning model**
We ceeate a model able to predict the miles per gallon of different car models based on specific feature of the cars. <br>
We'll use transfomr this model with Flask to be used as an End-Point.

In [ ]:
from werkzeug.wrappers import Request, Response
from flask import Flask

In [ ]:
app = Flask(__name__)

@app.route("/")              # display hello world in the web page
def hello():
    return "hello world"

if __name__=="__main__":     # run the application
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping       # it is the optioin to stop the training process if the NN is not getting any better
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

### **Load data**

In [ ]:
df = pd.read_csv(r'C:\Users\..\auto.mpg.cvs', na_values=['NA','?'])       # convert na
df.isnull().sum()                                                         # check null values
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())  # fill nulls with median

x = df[['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']]   # define the independent variables
y = df['mpg'].values                                                                              # convert the dependent variable to a numpy array

### **Split the data into train and test**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y test_size=0.25, random_state=42)

### **Define the Neural Network**

In [ ]:
model = Sequential()
# define first layer in the Neural network
model.add(Dense(25, input_dim=x.shape[1], # 25 neurons inside the first layer with in input 7 variables --> x.shape[1] --> (cylinders, etc.)
                    activation='relu'))   # define relu as activation function

# define a second hidden layer
model.add(Dense(10, activation='relu'))   # here is not necessary to define the input dimension --> is neeed only in the input layer

# define output layer
model.add(Dense(1))    # we define only 1 output --> mpg
                                          
# compile the model
model.compile(loss='mean_squared_error', optimizer='adam')    # use msr as loss function, use adaptive momentum to do gradient descent optimization


# define monitor:
# monitor the validation loss and 
# min_delta is an hyper-parameter --> is is the minimum 
# addition which we want the model to archive during training
# if min_delta=2e-3 is not reached --> stopp the leaning process
# patience=5 --> give the allowance to train 5 times 
# if after 5 time we not archive min_delta=1e-3 --> stop the process
# restore_best_weights=True --> if we not reachmin_delta=1e-3 --< resture the best archivment
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto', restore_best_weights=True) $


# traini the model
mode.fit(x_train, y=train, validation_data=(x_test, y_test), callbacks=[monitor], verbose=2, epochs=1000) 

# **Evaluating and saving the model**

In [ ]:
prediction = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(prediction, y_test))
print(f'the mean squared error between prediction and real mpg (y_test) is: {score}')

# **Save the model**

In [ ]:
os.chdir(r'C:\User...\Deploy')   # point to a specific folder
model.save(os.oath.join(os.getcwd()), "mpg_model.h5")